<a href="https://colab.research.google.com/github/Belebez/VK_test/blob/main/vk_task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Приветствую!



### Этап 1. Подготовка данных

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# прочитаем файлы
ratings = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/task_vk/ratings.csv')

In [24]:
movies = pd.read_csv('/content/drive/MyDrive/Colab_Notebooks/task_vk/movies.csv')

In [6]:
# Разбиваем данные на обучающую и тестовую выборки
train_ratings, test_ratings = train_test_split(ratings, test_size=0.2, random_state=42)

# Разделение на обучающую и тестовую выборки помогает оценить качество модели на реальных данных
# и сделать выводы о ее способности делать точные предсказания на новых примерах.

In [7]:
# Создаем словари для преобразования идентификаторов пользователей и фильмов в индексы
user_to_index = {u: i for i, u in enumerate(train_ratings['userId'].unique())}
movie_to_index = {m: i for i, m in enumerate(train_ratings['movieId'].unique())}

In [21]:
index_to_movie = {i: m for m, i in movie_to_index.items()}

In [8]:
# Преобразуем идентификаторы пользователей и фильмов в индексы
train_ratings['user_index'] = train_ratings['userId'].apply(lambda u: user_to_index[u])
train_ratings['movie_index'] = train_ratings['movieId'].apply(lambda m: movie_to_index[m])
test_ratings['user_index'] = test_ratings['userId'].apply(lambda u: user_to_index.get(u, -1))
test_ratings['movie_index'] = test_ratings['movieId'].apply(lambda m: movie_to_index.get(m, -1))

### Этап 2. Обучение модели

In [9]:
# Гиперпараметры модели
embedding_dim = 32
dropout_rate = 0.2
learning_rate = 0.001
batch_size = 512
epochs = 10

In [10]:
# Создадим модель

# Выбранная архитектура нейронной сети является базовой для задачи рекомендации фильмов на основе векторных представлений пользователей и фильмов. 

# Входной тензор для индексов пользователей
user_input = tf.keras.layers.Input(shape=(1,))
# Входной тензор для индексов фильмов
movie_input = tf.keras.layers.Input(shape=(1,))
# Слой эмбеддинга пользователей. Он преобразует индексы пользователей в плотные векторные представления фиксированного размера (32 в данном случае)
user_embedding = tf.keras.layers.Embedding(len(user_to_index), embedding_dim, name='user_embedding')(user_input)
# Слой эмбеддинга фильмов. Аналогично, он преобразует индексы фильмов в плотные векторные представления.
movie_embedding = tf.keras.layers.Embedding(len(movie_to_index), embedding_dim, name='movie_embedding')(movie_input)
# Соединение векторных представлений пользователей и фильмов в единый вектор. Здесь выполняется конкатенация векторов пользователей и фильмов
# по последней оси, чтобы объединить их вместе.
concatenated = tf.keras.layers.concatenate([user_embedding, movie_embedding], axis=-1)
# Dropout слой с коэффициентом отсева (dropout_rate = 0.2) применяется к соединенному вектору для регуляризации модели и снижения переобучения.
x = tf.keras.layers.Dropout(dropout_rate)(concatenated)
# Полносвязный слой с 128 нейронами и функцией активации ReLU. Он применяет линейное преобразование к входным данным и вводит нелинейность в модель.
x = tf.keras.layers.Dense(128, activation='relu')(x)
# Для дополнительной регуляризации.
x = tf.keras.layers.Dropout(dropout_rate)(x)
# Полносвязный слой с одним нейроном и линейной активацией. Он генерирует предсказанную оценку для рейтинга фильма.
output = tf.keras.layers.Dense(1, activation='linear')(x)

model = tf.keras.models.Model(inputs=[user_input, movie_input], outputs=output)



model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))

# Обоснование выбора MSE:

# 1. Соответствие задаче: MSE широко используется в задачах регрессии, где требуется предсказать числовые значения. 
# В случае рекомендательных систем, где оценивается рейтинг пользователя для фильма, MSE является естественным выбором, 
# поскольку она измеряет среднеквадратичное отклонение между предсказанными и фактическими значениями рейтинга.

# 2. Чувствительность к ошибкам: MSE позволяет учесть большие отклонения между предсказанными и фактическими значениями, 
# так как квадратичная функция увеличивает вес больших ошибок. Это полезно для рекомендательных систем, 
# где точность предсказания рейтинга имеет большое значение.

In [11]:
# Обучаем модель на обучающей выборке и оцениваем ее качество на валидационной выборке
history = model.fit(
    [train_ratings['user_index'], train_ratings['movie_index']],
    train_ratings['rating'],
    batch_size=batch_size,
    epochs=epochs,
    verbose=1,
    validation_split=0.2
)

Epoch 1/10
34692/34692 [==============================] - 278s 8ms/step - loss: 0.8128 - val_loss: 0.7310
Epoch 2/10
34692/34692 [==============================] - 250s 7ms/step - loss: 0.7196 - val_loss: 0.7096
Epoch 3/10
34692/34692 [==============================] - 281s 8ms/step - loss: 0.6983 - val_loss: 0.7010
Epoch 4/10
34692/34692 [==============================] - 263s 8ms/step - loss: 0.6840 - val_loss: 0.6907
Epoch 5/10
34692/34692 [==============================] - 251s 7ms/step - loss: 0.6717 - val_loss: 0.6838
Epoch 6/10
34692/34692 [==============================] - 252s 7ms/step - loss: 0.6608 - val_loss: 0.6799
Epoch 7/10
34692/34692 [==============================] - 258s 7ms/step - loss: 0.6518 - val_loss: 0.6733
Epoch 8/10
34692/34692 [==============================] - 251s 7ms/step - loss: 0.6445 - val_loss: 0.6700
Epoch 9/10
34692/34692 [==============================] - 249s 7ms/step - loss: 0.6384 - val_loss: 0.6695
Epoch 10/10
34692/34692 [=====================

In [12]:
# Оцениваем качество модели на тестовой выборке
test_loss = model.evaluate([test_ratings['user_index'], test_ratings['movie_index']], test_ratings['rating'])
print('Test MSE: ', test_loss)

173460/173460 [==============================] - 444s 3ms/step - loss: 0.6653
Test MSE:  0.6652815341949463


Модель показывает хорошие результаты в предсказании рейтингов пользователей для фильмов, среднеквадратичная ошибка составляет примерно 0.6653 по шкале от 1 до 10.

### Этап 3. Составление рекомендаций

In [13]:
# Получаем индексы всех фильмов
all_movie_indices = np.arange(len(movie_to_index))

In [14]:
# Выбираем случайного пользователя
user_index = np.random.choice(np.arange(len(user_to_index)))

In [15]:
# Отфильтровываем фильмы, которые пользователь уже оценил
rated_movies = train_ratings[train_ratings['user_index'] == user_index]['movie_index'].values
unrated_movies = np.setdiff1d(all_movie_indices, rated_movies)

In [16]:
# Создаем входные данные для предсказания рейтингов непросмотренных фильмов пользователем
user_indices = np.full(len(unrated_movies), user_index)
movie_indices = unrated_movies.reshape(-1, 1)

In [17]:
# Предсказываем рейтинги непросмотренных фильмов
predicted_ratings = model.predict([user_indices, movie_indices]).flatten()

# Использование flatten() позволяет преобразовать двумерный массив в одномерный массив, чтобы получить список предсказанных рейтингов,

1611/1611 [==============================] - 3s 2ms/step


In [27]:
# Получаем индексы фильмов с наивысшими предсказанными рейтингами
top_movies_indices = predicted_ratings.argsort()[::-1][:7]

In [28]:
# Получаем идентификаторы фильмов
top_movies_ids = [index_to_movie[i] for i in top_movies_indices]

In [29]:
# Выводим рекомендованные фильмы
recommended_movies = movies[movies['movieId'].isin(top_movies_ids)]
recommended_movies[['movieId', 'title', 'genres']]

,movieId,title,genres
315,318,"Shawshank Redemption, The (1994)",Crime|Drama
973,991,Michael Collins (1996),Drama
5554,5652,Claire of the Moon (1992),Drama|Romance
7345,7482,Enter the Dragon (1973),Action|Crime
11063,45648,Game 6 (2005),Comedy|Drama
15247,77414,"Last Song, The (2010)",Drama|Romance
21396,103210,Fullmetal Alchemist: The Sacred Star of Milos ...,Action|Adventure|Animation


Хорошего настроения!